## Setting
- 한글폰트 위해 나눔 글씨체 install 후 런타임 다시 시작
- CRNN은 괜찮지만 아래에서 다른 모듈로 predict 시에는 CPU보다 GPU가 빠름.(CPU로 실행해도 1~2분 내로 실행 됨)
- pj4_demo.ipynb 파일은 프로젝트 깃허브의 mmocr_custom 브랜치에 위치

In [ ]:
# 한글 plot 출력위해 실행 후 런타임 다시 시작
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

## Recognition use CRNN
- 논문: https://arxiv.org/abs/1507.05717
- 논문 github: https://github.com/bgshih/crnn#train-a-new-model
- python ver github: https://github.com/meijieru/crnn.pytorch

demo 예측으로 사용하는 pre-trained 모델은 숫자와 영어 소문자만 구분할 수 있는 모델만 제공.

실제 논문을 보면 여러 데이터셋을 학습시킨 결과들이 나오고, 악보인식도 모델이 잘 해냄을 보임.

In [ ]:
# 모듈 import
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
!pip install lmdb
import lmdb

In [ ]:
# CRNN을 python pytorch로 구현한 github clone
!git clone https://github.com/meijieru/crnn.pytorch

In [ ]:
# 미리 학습된 pth 파일 받아오기
%cd /content/crnn.pytorch/data
!wget https://github.com/PaulowniaTree/mini_project_4/raw/mmocr_custom/demo/crnn.pth

In [ ]:
# 첫번째 demo 이미지 읽기
img_path = '/content/crnn.pytorch/data/demo.png'
plt.imshow(np.array(Image.open(img_path).convert('RGB')))
plt.axis('off'); plt.show()
%cd /content/crnn.pytorch
!python /content/crnn.pytorch/demo.py

In [ ]:
# 또 다른 이미지 가져오기
%cd /content/crnn.pytorch/data
!wget https://github.com/bgshih/crnn/raw/master/data/demo2.jpg
# /content/crnn.pytorch/demo.py의 demo.png를 demo2.jpg로 변경

In [ ]:
# 두번째 demo 이미지 읽기
img_path = '/content/crnn.pytorch/data/demo2.jpg'
plt.imshow(np.array(Image.open(img_path).convert('RGB')))
plt.axis('off'); plt.show()
%cd /content/crnn.pytorch
!python /content/crnn.pytorch/demo.py

## OCR use MMOCR
- detection, recognition을 각각 예측 또는 한 번에 예측 가능
- 언어 설정은 영어와 중국어
- 다양한 모델을 바꾸어가면서 실행시켜 볼 수 있음.

In [ ]:
!pip install -U openmim
!mim install mmengine
!mim install 'mmcv>=2.0.0rc1'
!mim install 'mmdet>=3.0.0rc0'
%cd /content
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -v -e .

In [ ]:
from mmocr.apis import MMOCRInferencer
import mmcv
import matplotlib.pyplot as plt

In [ ]:
# recognition
infer = MMOCRInferencer(rec='svtr-small')
result = infer('demo/demo_text_recog.jpg', return_vis=True)
print(result['predictions'])
plt.imshow(result['visualization'][0])
plt.axis('off'); plt.show()

In [ ]:
# detection
infer = MMOCRInferencer(det='dbnetpp')
result = infer('demo/demo_text_det.jpg', return_vis=True)
plt.figure(figsize=(9, 16))
plt.imshow(result['visualization'][0])
plt.axis('off'); plt.show()

In [ ]:
# detection, recognition
infer = MMOCRInferencer(det='dbnetpp', rec='svtr-small')
result = infer('demo/demo_text_ocr.jpg', return_vis=True)
plt.figure(figsize=(9, 16))
plt.imshow(result['visualization'][0])
plt.axis('off'); plt.show()

In [ ]:
# detection, recognition, key information extract(kie)
infer = MMOCRInferencer(det='dbnetpp', rec='svtr-small', kie='SDMGR')
result = infer('demo/demo_kie.jpeg', save_vis=True)
predicted_img = mmcv.imread('results/vis/demo_kie.jpg')
plt.figure(figsize=(18, 32))
plt.imshow(mmcv.bgr2rgb(predicted_img))
plt.axis('off'); plt.show()

## OCR use EasyOCR
- detection과 recognition 과정이 연결되어 있어 한 번에 예측
- 언어 설정 가능
- 한글 언어 예측이 조금 약한 듯 함(naver와 kakao에서 만든 모듈은 한글 인식이 더 잘 된다는 평이 있으나 직접 비교는 해보지 않음)
- craft, dbnet을 dectection으로 사용, recognition에는 CRNN을 기반한 모델 사용(CNN-BiLSTM-Att)

In [ ]:
from matplotlib import pyplot as plt
!pip install easyocr
from easyocr import Reader

In [2]:
import cv2
import requests
import numpy as np
from PIL import ImageFont, ImageDraw, Image

In [ ]:
# image 가져오기
%cd /content/
!wget https://github.com/PaulowniaTree/mini_project_4/raw/jinn0135/Temp/temp_Sign/000D6985A1C98F0549F53FEE4FA556B8.jpg
img_path = '/content/000D6985A1C98F0549F53FEE4FA556B8.jpg'
img = np.array(Image.open(img_path).convert('RGB'))

In [ ]:
# 언어설정, 예측
langs = ['ko', 'en']
print("[INFO] OCR'ing input image...")
reader = Reader(lang_list=langs, gpu=True)
results = reader.readtext(img)
results

In [ ]:
# polygon 없이 text만 보기
simple_results = reader.readtext(img, detail=0)
simple_results

In [ ]:
trans_img = img.copy()
def putText(cv_img, text, x, y, color=(0,0,0), font_s=22):
    font = ImageFont.truetype('/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf', font_s)
    img = Image.fromarray(cv_img)
    draw = ImageDraw.Draw(img)
    draw.text((x, y), text, font=font, fill=color)
    cv_img = np.array(img)
    return cv_img

for (bbox, text, prob) in results:
    # print('{:.4f}: {}'.format(prob, text))
    pts = [(int(p[0]), int(p[1])) for p in bbox]
    cv2.rectangle(trans_img, pts[0], pts[2], (0,255,0), 2)
    trans_img = putText(trans_img, text, pts[0][0], pts[0][1]-60, (0,255,0), 50)

fig, ax = plt.subplots(1,2, figsize=(18,32))
ax[0].imshow(img)
ax[1].imshow(trans_img)
ax[0].axis('off'); ax[1].axis('off')
plt.show()